## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import pytz
import glob
import json
import requests
import pandas as pd
import requests
from datetime import datetime
from slugify import slugify

## Download

In [6]:
POWER_BI_QUERY_URL = (
    "https://wabi-us-gov-iowa-api.analysis.usgovcloudapi.net/public/reports/querydata"
)
REQUEST_PARAMS = (("synchronous", "true"), ("Connection", "keep-alive"))
REQUEST_BODY = '{"version":"1.0.0","queries":[{"Query":{"Commands":[{"SemanticQueryDataShapeCommand":{"Query":{"Version":2,"From":[{"Name":"z","Entity":"zipcodes_cases","Type":0}],"Select":[{"Column":{"Expression":{"SourceRef":{"Source":"z"}},"Property":"zipcode"},"Name":"zipcodes_cases.zipcode"},{"Column":{"Expression":{"SourceRef":{"Source":"z"}},"Property":"cases_per_100k"},"Name":"Sum(zipcodes_cases.cases_per_100k)"},{"Aggregation":{"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"z"}},"Property":"cases"}},"Function":0},"Name":"Sum(zipcodes_cases.cases)"}],"OrderBy":[{"Direction":2,"Expression":{"Column":{"Expression":{"SourceRef":{"Source":"z"}},"Property":"cases_per_100k"}}}]},"Binding":{"Primary":{"Groupings":[{"Projections":[0,1,2]}]},"DataReduction":{"DataVolume":3,"Primary":{"Window":{"Count":500}}},"Version":1}}}]},"CacheKey":"{\\"Commands\\":[{\\"SemanticQueryDataShapeCommand\\":{\\"Query\\":{\\"Version\\":2,\\"From\\":[{\\"Name\\":\\"z\\",\\"Entity\\":\\"zipcodes_cases\\",\\"Type\\":0}],\\"Select\\":[{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"z\\"}},\\"Property\\":\\"zipcode\\"},\\"Name\\":\\"zipcodes_cases.zipcode\\"},{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"z\\"}},\\"Property\\":\\"cases_per_100k\\"},\\"Name\\":\\"Sum(zipcodes_cases.cases_per_100k)\\"},{\\"Aggregation\\":{\\"Expression\\":{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"z\\"}},\\"Property\\":\\"cases\\"}},\\"Function\\":0},\\"Name\\":\\"Sum(zipcodes_cases.cases)\\"}],\\"OrderBy\\":[{\\"Direction\\":2,\\"Expression\\":{\\"Column\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"z\\"}},\\"Property\\":\\"cases_per_100k\\"}}}]},\\"Binding\\":{\\"Primary\\":{\\"Groupings\\":[{\\"Projections\\":[0,1,2]}]},\\"DataReduction\\":{\\"DataVolume\\":3,\\"Primary\\":{\\"Window\\":{\\"Count\\":500}}},\\"Version\\":1}}}]}","QueryId":"","ApplicationContext":{"DatasetId":"38141830-4b1b-4309-b51e-da91c50f22ee","Sources":[{"ReportId":"cef91e37-b89d-4967-8130-dc528ff1d964"}]}}],"cancelQueries":[],"modelId":276737}'
DATE_REQUEST_BODY = '{"version":"1.0.0","queries":[{"Query":{"Commands":[{"SemanticQueryDataShapeCommand":{"Query":{"Version":2,"From":[{"Name":"h","Entity":"health-summary","Type":0}],"Select":[{"Measure":{"Expression":{"SourceRef":{"Source":"h"}},"Property":"updated"},"Name":"health-summary.updated"}]},"Binding":{"Primary":{"Groupings":[{"Projections":[0]}]},"DataReduction":{"DataVolume":3,"Primary":{"Top":{}}},"Version":1}}}]},"CacheKey":"{\\"Commands\\":[{\\"SemanticQueryDataShapeCommand\\":{\\"Query\\":{\\"Version\\":2,\\"From\\":[{\\"Name\\":\\"h\\",\\"Entity\\":\\"health-summary\\",\\"Type\\":0}],\\"Select\\":[{\\"Measure\\":{\\"Expression\\":{\\"SourceRef\\":{\\"Source\\":\\"h\\"}},\\"Property\\":\\"updated\\"},\\"Name\\":\\"health-summary.updated\\"}]},\\"Binding\\":{\\"Primary\\":{\\"Groupings\\":[{\\"Projections\\":[0]}]},\\"DataReduction\\":{\\"DataVolume\\":3,\\"Primary\\":{\\"Top\\":{}}},\\"Version\\":1}}}]}","QueryId":"","ApplicationContext":{"DatasetId":"38141830-4b1b-4309-b51e-da91c50f22ee","Sources":[{"ReportId":"cef91e37-b89d-4967-8130-dc528ff1d964"}]}}],"cancelQueries":[],"modelId":276737}'

In [7]:
headers = {
    "Host": "wabi-us-gov-iowa-api.analysis.usgovcloudapi.net",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https",
    "ActivityId": "9cd51346-a449-f3a5-dd4b-24ecf8d4bea1",
    "RequestId": "4e47cd43-3ecb-73c9-c9d9-b01a0f1193d2",
    "X-PowerBI-ResourceKey": "03f5c7f8-6069-4a12-8a25-542155ec7b99",
    "Content-Type": "application/json;charset=UTF-8",
    "Origin": "https://app.powerbigov.us",
    "DNT": "1",
    "Connection": "keep-alive",
}
json_data = json.loads(REQUEST_BODY)

In [8]:
date_headers = {
    "Host": "wabi-us-gov-iowa-api.analysis.usgovcloudapi.net",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9,da;q=0.8",
    "Referer": "https://app.powerbigov.us/",
    "ActivityId": "bb9764f2-4254-dcb7-ba32-341f4e87dba7",
    "RequestId": "ae8d4895-2f30-4d5f-6d92-1399d2c76c6e",
    "X-PowerBI-ResourceKey": "03f5c7f8-6069-4a12-8a25-542155ec7b99",
    "Content-Length": "1047",
    "Content-Type": "application/json;charset=UTF-8",
    "DNT": "1",
    "Origin": "https://app.powerbigov.us",
    "Pragma": "no-cache",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "cross-site",
    "Connection": "keep-alive",
}

In [9]:
response = requests.post(
    POWER_BI_QUERY_URL,
    params=REQUEST_PARAMS,
    headers=headers,
    json=json_data,
    verify=False,
)

/home/palewire/.local/share/virtualenvs/california-coronavirus-scrapers-s_ZEjlcw/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wabi-us-gov-iowa-api.analysis.usgovcloudapi.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [10]:
data = response.json()

## Parse

In [11]:
ds = data["results"][0]["result"]["data"]["dsr"]["DS"][0]

In [12]:
zip_codes = ds["ValueDicts"]["D0"]

In [13]:
rows = ds["PH"][0]["DM0"]

In [15]:
confirmed_cases = []

cases = 0

for row in rows:
    # in some cases, data are encoded in separate keys
    keys = row.keys()
    # pull out the list containing values for most rows
    cells = row["C"]
    # first value in the list is expected to be an iterator/index
    index = cells[0]
    # if this key is present, the value for this place is identical to the previous row
    if "R" in keys:
        pass
    # if this key is present, the value is found in the second position of the list
    elif "Ø" in keys:
        cases = cells[1]
    # every other row has its value in the third position of the list
    else:
        cases = cells[2]
    # append values to master list
    confirmed_cases.append(
        (
            index,
            cases,
        )
    )

In [16]:
matched = [(zip_codes[index], cases) for index, cases in confirmed_cases]

In [17]:
date_data = json.loads(DATE_REQUEST_BODY)
response = requests.post(
    POWER_BI_QUERY_URL,
    params=REQUEST_PARAMS,
    headers=date_headers,
    json=date_data,
    verify=False,
)
data = response.json()

/home/palewire/.local/share/virtualenvs/california-coronavirus-scrapers-s_ZEjlcw/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wabi-us-gov-iowa-api.analysis.usgovcloudapi.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [18]:
raw_date_string = data["results"][0]["result"]["data"]["dsr"]["DS"][0]["PH"][0]["DM0"][
    0
]["M0"]
raw_date_string

'Data updated on 09/02/2021'

In [20]:
raw_date = raw_date_string.replace("Data updated on", "").strip()
county_date = datetime.strptime(raw_date, "%m/%d/%Y")

In [21]:
df = pd.DataFrame(matched, columns=["zip", "confirmed_cases"])

In [22]:
df.insert(0, "county", "Long Beach")

In [23]:
df["county_date"] = county_date

In [24]:
df["area"] = df["zip"] + ": Long Beach"

In [25]:
df = df[["county", "area", "confirmed_cases", "county_date", "zip"]]

## Vet

In [26]:
try:
    assert not len(df) > 11
except AssertionError:
    raise AssertionError("Long Beach's zip code scraper has extra rows")

In [27]:
try:
    assert not len(df) < 11
except AssertionError:
    raise AssertionError("Long Beach's zip code scraper is missing rows")

## Export

Set date

In [28]:
tz = pytz.timezone("America/Los_Angeles")

In [29]:
today = datetime.now(tz).date()

In [30]:
slug = "long-beach"

In [31]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [32]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [33]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [34]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [35]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)